In [52]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Теоретична частина
## Поліноми Лагера 1.1
Поліноми Лагера є розв'язками диференціального рівняння:
$$xy''+(1-x)y'+ny=0$$

Поліноми Лагера визначаються формулою:
$$L_{n}(x)={\frac {e^{x}}{n!}}{\frac {d^{n}}{dx^{n}}}\left(e^{-x}x^{n}\right)$$

## Функції лагера 1.2
Функції Лагера визначаються формулою:
$$l_n(t)={\sqrt\sigma}L_n(t){e^{-t\frac{{\beta}}{2}}}$$
де $\sigma$ і $\beta$ є параметрами. І $0\le\beta\le\sigma \quad \beta, \sigma \in \R$

Ми можеио визначити функції Лагера рекурсивно:

$$\begin{cases}
\tag{1}
l_0(t)=\sqrt\sigma{e^{-t\frac{{\beta}}{2}}}\\
{l_1(t)=\sqrt\sigma{(1-\sigma{t})}{e^{-t\frac{{\beta}}{2}}}}\\
l_{n}(t)=\frac{2n-1-\sigma{t}}{n}l_{n-1}- \frac{n-1}{n}l_{n-2}
\end{cases}
$$

In [53]:
def laguerre(t, n, beta=2, sigma=4):
    if beta < 0 or beta > sigma:
        raise ValueError("Wrong parameters")
    

    lpp = np.sqrt(sigma)*np.exp(-beta*t/2)

    lp = np.sqrt(sigma)*(1 - sigma*t)*np.exp(-beta*t/2)

    if n == 0:
        return lpp
    if n == 1:
        return lp

    for i in range(2, n+1):
        temp = lp
        lp = (2*i -1 -sigma*t)*lp/i - (i-1)*lpp / i
        lpp = temp
     
    return lp

Приклад використання функції Лагера з параметрами $\sigma=4$ і $\beta=2$ для $n=2$ і $t=1$:

In [54]:
laguerre(1, 2, beta=2, sigma=4)

0.7357588823428847

Табуляція функції Лагера:

In [55]:
def tabulate_laguerre(T, n, beta, sigma):
    t = np.linspace(0, T, 100)
    results = laguerre(t, n, beta, sigma)
    df = pd.DataFrame({'t': t, 'l': results})
    return df.round(5)

In [56]:
tabulate_laguerre(10, 2, 2, 4).head()

,t,l
0,0.00000,2.00000
1,0.10101,0.49453
2,0.20202,-0.47336
3,0.30303,-1.01868
4,0.40404,-1.23687


### Експеримент
Легко побачити що функції Лагера прямують до нуля при $t\to\infty$. Тепер ми спробуємо знайти значення $t$ при якому $l_n(t)<10^{-3}$<br>${\forall}n\in[1,N]$

In [57]:
def experiment(T, beta, sigma, epsilon=1e-3, N=20):
    t = np.linspace(0, T, 1000)
    n = range(1, N+1)
    result = None
    for i in t:
        flag = True
        for j in n:
            if abs(laguerre(i, j, beta, sigma)) > epsilon:
                flag = False
                break
        if flag and result is None:
            result = i
    

    cols = {"t" : t}
    for j in n:
        cols[f"n={j}"] = laguerre(t, j, beta, sigma)

    df = pd.DataFrame(cols)

    return result, df.round(5)


In [58]:
r, df = experiment(100, 2, 4)
r

79.07907907907908

Результатом для $N=20$ є:
<br>
$t\approx79$ - остання точка при якій $l_n(t)<10^{-3}$
<br>
Це можна буде використати для визначення верхньої межі інтегрування в подальшому.


## Перетворення Лагера 1.3
Перетворення Лагера ставить функції $f(x)$ відповідний вектор значень $f := (f_0, f_1, \dots, f_k, \dots)$, де $f_k$ визначається як:

$$\begin{equation}

f_k=\int_0^\infty f(t)l_k(t)e^{-\alpha{t}}dt
\tag{2}
\end{equation}$$
$$\alpha = \sigma - \beta$$

Для початку визначимо функцію для чисельного інтегрування методом прямокутників:

In [59]:
def quad(f, a, b, N=10000):   
    x = np.linspace(a, b, N)
    s = sum([f(i) for i in x])
    return s*abs(b-a)/N

Після цього визначаємо функцію для обрахунку перетворення Лагера $(2)$:

In [60]:
def laguerre_transformation(f, n, beta=2, sigma=4):
    def integrand(t):
        return f(t)*laguerre(t, n, beta, sigma)*np.exp(-t*(sigma-beta))
    b = experiment(100, beta, sigma)[0]

    return quad(integrand, 0, b)

In [61]:
def tabulate_transformation(f, N, beta, sigma):
    results = [laguerre_transformation(f, n, beta, sigma) for n in range(N+1)]
    
    return results

Дана функція:
$f(t)=$
$$
\begin{cases}
\sin{(t-\frac{\pi}{2})}+1 & \text{if } t\in[0,2\pi]\\
0 & \text{інакше}
\end{cases}$$

In [62]:
def f(t):
    if t >= 0 and t <= 2*np.pi:
        return np.sin(t-np.pi/2) + 1
    else:
        return 0

І застосовуємо перетворення Лагера:

In [63]:
tabulate_transformation(f, 20, 2, 4)

[0.06665999946809152,
 -0.1822039881310192,
 0.17805610913078898,
 -0.0742826695000306,
 0.007262784325811692,
 0.007587430478937864,
 -0.003096494945065475,
 -0.0006148703444646362,
 0.0007994250066752776,
 -2.5850153812011465e-05,
 -0.00023592602139502894,
 5.256960095504625e-05,
 9.381390277832641e-05,
 -3.0676683574269335e-05,
 -5.2658350956100556e-05,
 1.097528013689054e-05,
 3.616172697533956e-05,
 4.5476548832641705e-06,
 -2.3361494137766216e-05,
 -1.4823620942386276e-05,
 8.910978104645316e-06]

## Обернене перетворення Лагера 1.4
Обернене перетворення Лагера ставить у відповідність вектору значень $h := (h_0, h_1, \dots, h_k, \dots)$ функцію $h(t)$
$$\begin{equation}
\tag{3}
h(t)=\sum_{k=0}^\infty h_kl_k(t)
\end{equation}$$
де $l_k(t)$ - функція Лагера.

In [64]:
def reversed_laguerre_transformation(h_list, t, beta=2, sigma=4):
    result_sum = 0

    h_list_new = list(filter(lambda x: x != 0, h_list))

    for i in range(len(h_list_new)):
        result_sum += h_list_new[i]*laguerre(t, i, beta, sigma)
    
    return result_sum

In [65]:
transfomormed_temp = tabulate_transformation(f, 20, 2, 4)
reversed_laguerre_transformation(transfomormed_temp, np.pi, 2, 4)

1.9997392005657693

# Графічне відображення результатів
## Функції Лагера

In [66]:
def plot_laguerre(T, N, beta=2, sigma=4):
    plt.close("all")
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()
    for n in range(N+1):
        laguerre_values = tabulate_laguerre(T, n, beta, sigma)
        ax.plot(laguerre_values['t'], laguerre_values['l'], label=f"n={n}", linewidth=2.0, alpha=0.7)
    
    ax.set_xlabel("t")
    ax.set_ylabel("l(t)")
    ax.set_title("Функції Лагера")
    fig.legend(loc='lower center', ncol=5)
    plt.show()


In [67]:

t_slider = widgets.FloatSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='T:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
beta_slider = widgets.FloatSlider(
    value=2,
    min=0,
    max=10,
    step=0.1,
    description='beta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
sigma_slider = widgets.FloatSlider(
    value=4,
    min=beta_slider.value,
    max=10,
    step=0.1,
    description='sigma:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

N_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='N:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
def update_sigma_min(change):
    sigma_slider.min = change['new']

beta_slider.observe(update_sigma_min, 'value')

output = widgets.Output()

def update_plot(change):
    with output:
        output.clear_output(wait=True)
        plot_laguerre(t_slider.value, N_slider.value, beta_slider.value, sigma_slider.value)

t_slider.observe(update_plot, 'value')
N_slider.observe(update_plot, 'value')
beta_slider.observe(update_plot, 'value')
sigma_slider.observe(update_plot, 'value')

box = widgets.VBox([t_slider, beta_slider, sigma_slider, N_slider, output])

display(box)

## Перетворення Лагера

In [68]:
def plot_transformation(f, n, beta=2, sigma=4):
    plt.close("all")
    fig = plt.figure(figsize=(5, 5))
    ax = fig.gca()
    transform_values = tabulate_transformation(f, n, beta, sigma)
    ax.bar(range(n+1), transform_values, alpha=0.7, edgecolor='black', width=1)

    ax.set_xlabel("n")
    ax.set_ylabel("f_n")
    ax.set_title("Коефіцієнти перетворення")
    ax.set_xticks(range(n+1))
    fig.tight_layout()
    plt.axhline(0, color='black')
    plt.show()

In [69]:

beta_slider = widgets.FloatSlider(
    value=2,
    min=0,
    max=10,
    step=0.1,
    description='beta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
sigma_slider = widgets.FloatSlider(
    value=4,
    min=beta_slider.value,
    max=10,
    step=0.1,
    description='sigma:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

N_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='N:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
def update_sigma_min(change):
    sigma_slider.min = change['new']

beta_slider.observe(update_sigma_min, 'value')

output = widgets.Output()

def update_plot(change):
    with output:
        output.clear_output(wait=True)
        plot_transformation(f, N_slider.value, beta_slider.value, sigma_slider.value)

N_slider.observe(update_plot, 'value')
beta_slider.observe(update_plot, 'value')
sigma_slider.observe(update_plot, 'value')

box = widgets.VBox([beta_slider, sigma_slider, N_slider, output])

display(box)

## Оберенене перетворення Лагера

In [70]:
def plot_tranformations(f, n, beta=2, sigma=4, t1=0, t2=2*np.pi):
    plt.close("all")
    transform_values = tabulate_transformation(f, n, beta, sigma)
    reversed_transform_values = [reversed_laguerre_transformation(transform_values, t, beta, sigma) for t in np.linspace(t1, t2, 1000)]
    correct_values = [f(t) for t in np.linspace(t1, t2, 1000)]

    fig = plt.figure(figsize=(10, 10))
    ax = fig.subplots(2, 1)
    ax[0].bar(range(n+1), transform_values, alpha=0.7, edgecolor='black')

    ax[0].set_xlabel("n")
    ax[0].set_ylabel("f_n")
    ax[0].set_title("Коефіцієнти перетворення")
    ax[0].set_xticks(range(n+1))
    fig.tight_layout()
    ax[0].axhline(0, color='black')

    ax[1].plot(np.linspace(t1, t2, 1000), reversed_transform_values, alpha=0.7, linewidth=2.0, label="Отримана функція")
    ax[1].plot(np.linspace(t1, t2, 1000), correct_values, alpha=0.7, linewidth=2.0, linestyle="--",label="Початкова функція")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("f(t)")
    ax[1].set_title("Обернене перетворення")
    ax[1].legend(loc='lower center', ncol=2)
    plt.show()

In [71]:
def create_transformation_widget(func, t0, t1):    
    beta_slider = widgets.FloatSlider(
        value=2,
        min=0,
        max=10,
        step=0.1,
        description='beta:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )
    sigma_slider = widgets.FloatSlider(
        value=4,
        min=beta_slider.value,
        max=10,
        step=0.1,
        description='sigma:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

    N_slider = widgets.IntSlider(
        value=10,
        min=0,
        max=100,
        step=1,
        description='N:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    def update_sigma_min(change):
        sigma_slider.min = change['new']

    beta_slider.observe(update_sigma_min, 'value')

    output = widgets.Output()

    def update_plot(change):
        with output:
            output.clear_output(wait=True)
            plot_tranformations(func, N_slider.value, beta_slider.value, sigma_slider.value, t0, t1)

    N_slider.observe(update_plot, 'value')
    beta_slider.observe(update_plot, 'value')
    sigma_slider.observe(update_plot, 'value')

    box = widgets.VBox([beta_slider, sigma_slider, N_slider, output])

    display(box)

In [72]:
create_transformation_widget(f, 0, 2*np.pi)

Врешті, порівнявши початкову і отриману функцію, ми бачимо що вони майже ідентичні. Це означає що обернене перетворення, і будь-яка інша функція, працює правильно.

Продемонструємо результат і на іншій функції:
$$
\begin{cases}
\cos{t} + 1&  \text{if } t\in[0,\frac{\pi}{2}]\\
1 & \text{інакше}
\end{cases}
$$

In [73]:
def custom_func(t):
    if t >= 0 and t <= np.pi/3:
        return np.tan(t)
    else:
        return 2



In [74]:
create_transformation_widget(custom_func, 0, np.pi/3)

Бачимо що результат також апроксимує початкову функцію.

Отже, під час роботи над заваданням ми навчилися використовувати мову програмування Python і її бібліотеки, такі як: numpy, matplotlib, scipy; для імплементації функцій Лагера, перетворення Лагера і оберненого перетворення Лагера.